# VAEを用いた数字合成

TensorFlowチュートリアルのコードを一部利用

## ライブラリのインストール

In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.data import Dataset
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Conv2DTranspose, Reshape
from tensorflow.keras.optimizers import Adam

# 学習

## モデルの設定

In [49]:
class CondVAE(Model):
    def __init__(self, n_zdim, bs):
        super(CondVAE, self).__init__()
        self.n_zdim = n_zdim
        self.bs = bs
        self.encoder = self.encoder_model(self.n_zdim)
        self.decoder = self.decoder_model(self.n_zdim) 

    # エンコーダモデルの定義
    def encoder_model(self, n_zdim):
        encoder_in = Input(shape=(28, 28, 11)) # 教師ラベルチャンネルを含む
        h = Conv2D(32, (3, 3), strides=(2, 2), activation='relu')(encoder_in)
        h = Conv2D(64, (3, 3), strides=(2, 2), activation='relu')(h)
        h = Flatten()(h)
        z_mu = Dense(n_zdim, name='z_mu')(h) # 平均
        z_logvar = Dense(n_zdim, name='z_logvar')(h) # 対数分散
        return Model(inputs=encoder_in, outputs=[z_mu, z_logvar]) # 潜在変数の平均・分散，数字分類結果

    # デコーダモデルの定義
    def decoder_model(self, n_zdim): # 潜在ベクトル次元+数字のone-hotベクトル(10次元) 
        latent_in = Input(shape=(n_zdim+10, )) # 教師ラベル（ワンホット連結済み）
        h = Dense(7*7*32, activation='relu')(latent_in)
        h = Reshape(target_shape=(7, 7, 32))(h)
        h = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')(h)
        h = Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu')(h)
        reconst = Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', activation='linear', name='output')(h)
        return Model(inputs=latent_in, outputs=reconst)
        
    # 再パラメタライゼーショントリック
    def reparameterization(self, z_mu, z_logvar):
        epsilon = tf.random.normal(shape=tf.shape(z_logvar), mean=0, stddev=1.0) # epsilon生成
        return z_mu + epsilon * tf.exp(0.5 * z_logvar)

    # エンコード処理
    def encode(self, x, y): 
        # 教師ラベルは(28, 28, 10)のテンソルに変換してオリジナル画像にチャンネルを合成する
        bs = x.shape[0] # バッチサイズを取得
        one = tf.ones((28, 28, 1)) # すべての要素が1の(28, 28, 1)のテンソルを作る
        onehot = tf.reshape(tf.one_hot(y, 10), [self.bs, 1, 1, 10]) # (1, 1)の画像形式のone-hotベクトルを用意する
        onehot = one * onehot # (28, 28, 10)の画像one-hotベクトル。該当する教師ラベルのチャンネルのみ1，それ以外のチャンネルは0になる。例えば，ラベルが3の場合，3チャンネル目はすべて1，それ以外のチャンネルは0となる                
        x = tf.concat([x, onehot], axis=3) # 入力画像xとone-hotベクトルを結合して(bs, 28, 28, 11)のデータにする
        mean, logvar = self.encoder(x)
        return mean, logvar

    # デコード処理
    def decode(self, z, y):
        # 教師ラベルは(10, )のワンホットベクトルして潜在ベクトルとcancatする
        z = tf.concat([z, tf.one_hot(y, 10)], axis=1) 
        return self.decoder(z)

    # 確率密度関数
    def log_normal_pdf(self, sample, mean, logvar, raxis=1):
        log2pi = tf.math.log(2. * np.pi)
        return tf.reduce_sum(
            -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi), axis=raxis)

    # 損失計算・誤差逆伝播関数をオーバーライド    
    def train_step(self, data):        
        x, y = data
        with tf.GradientTape() as tape:
            z_mu, z_logvar = self.encode(x, y) # 入力画像の潜在ベクトルを計算
            z = self.reparameterization(z_mu, z_logvar) # 潜在ベクトルの生成
            x_logit = self.decode(z, y) # 潜在ベクトルから画像を生成
            """
            損失計算（TensorFlowチュートリアルのコードを利用）
            """
            cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
            logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
            logpz = self.log_normal_pdf(z, 0., 0.)
            logqz_x = self.log_normal_pdf(z, z_mu, z_logvar)
            total_loss = -tf.reduce_mean(logpx_z + logpz - logqz_x)

        gradients = tape.gradient(total_loss, self.trainable_variables) # 勾配計算
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables)) # 誤差逆伝播
        return {'loss': total_loss}

## 訓練データの読み込み

訓練データの画像のみ利用

In [50]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() # 訓練データの画像のみ利用
x_train = np.expand_dims(np.float32(x_train), -1) / 255. # チャンネルを増やして，float32に変換して正規化
y_train = np.int32(y_train)

## データの変換

tf.Tensor形式のデータセットの準備

バッチサイズ:50

In [51]:
train_ds = Dataset.from_tensor_slices((x_train, y_train))    
train_ds = train_ds.shuffle(buffer_size=len(train_ds))
train_ds = train_ds.repeat(1)
train_ds = train_ds.batch(50)#バッチサイズ:50

## モデルの作成

モデルの作成(入力:100次元，バッチサイズ:50)

最適化関数の設定(初期学習率:0.0001)

In [52]:
# モデルの作成(入力:100次元，バッチサイズ:50)
model = CondVAE(100, 50) 
# 最適化関数の設定(初期学習率:0.0001)
model.compile(optimizer=Adam(learning_rate=0.0001)) 

## 学習の実行

エポック数:20

In [53]:
model.fit(train_ds, epochs=20) 

Epoch 1/20
1200/1200 [==============================] - 112s 89ms/step - loss: 212.6839
Epoch 2/20
1200/1200 [==============================] - 119s 99ms/step - loss: 145.6498
Epoch 3/20
1200/1200 [==============================] - 114s 95ms/step - loss: 131.5717
Epoch 4/20
1200/1200 [==============================] - 116s 97ms/step - loss: 124.1629
Epoch 5/20
1200/1200 [==============================] - 121s 100ms/step - loss: 118.9274
Epoch 6/20
1200/1200 [==============================] - 117s 97ms/step - loss: 115.0596
Epoch 7/20
1200/1200 [==============================] - 118s 98ms/step - loss: 112.2002
Epoch 8/20
1200/1200 [==============================] - 115s 95ms/step - loss: 110.1127
Epoch 9/20
1200/1200 [==============================] - 114s 94ms/step - loss: 108.4143
Epoch 10/20
1200/1200 [==============================] - 112s 93ms/step - loss: 107.0662
Epoch 11/20
1200/1200 [==============================] - 114s 95ms/step - loss: 105.9394
Epoch 12/20
1200/1200 [======

## モデルの保存

In [54]:
model_enc = os.path.join('model', 'MNIST_SUPVAE_ENC')
model_dec = os.path.join('model', 'MNIST_SUPVAE_DEC')
model.encoder.save_weights(model_enc) # 訓練済みパラメータの保存
model.decoder.save_weights(model_dec)